# Tutorial para cortar imagen en una región

Se explica como definir los límites y cortar la imagen.

In [1]:
from netCDF4 import Dataset
from pyproj import Proj
import numpy as np

## 1. Selección de los datos

### Primero leamos una imagen

Tomaré la del ejemplo de descarga

In [2]:
ds = Dataset('OR_ABI-L2-TPWF-M6_G16_s20201251700143_e20201251709451_c20201251710417.nc')

In [3]:
ds.variables.keys()

dict_keys(['TPW', 'DQF_Overall', 'DQF_Retrieval', 'DQF_SkinTemp', 't', 'y', 'x', 'time_bounds', 'goes_imager_projection', 'y_image', 'y_image_bounds', 'x_image', 'x_image_bounds', 'nominal_satellite_subpoint_lat', 'nominal_satellite_subpoint_lon', 'nominal_satellite_height', 'geospatial_lat_lon_extent', 'outlier_pixel_count', 'minimum_total_precipitable_water', 'maximum_total_precipitable_water', 'mean_total_precipitable_water', 'standard_deviation_total_precipitable_water', 'algorithm_dynamic_input_data_container', 'processing_parm_version_container', 'algorithm_product_version_container', 'retrieval_local_zenith_angle', 'quantitative_local_zenith_angle', 'retrieval_local_zenith_angle_bounds', 'quantitative_local_zenith_angle_bounds', 'solar_zenith_angle', 'solar_zenith_angle_bounds', 'latitude', 'latitude_bounds', 'sounding_emissive_wavelengths', 'sounding_emissive_band_ids', 'percent_uncorrectable_L0_errors', 'percent_uncorrectable_GRB_errors', 'total_attempted_retrievals', 'mean_ob

Como vemos, se guardan muchas variables, de las cuales solamente usaremos algunas. En mi caso, las únicas que he necesitado son las coordenadas **x** e **y**, la variable **goes_imager_projection** (que contiene información relevante, como altura y longitud del satélite); la medida como tal, que en este caso es **TPW** (Total Precipitable Water); y de pronto el tiempo **t** (que en realidad yo lo tomo del nombre de la imagen).

Recordemos que para ver qué es una variable se puede hacer lo siguiente:

In [4]:
ds.variables['TPW']

<class 'netCDF4._netCDF4.Variable'>
int16 TPW(y, x)
    _FillValue: -1
    long_name: ABI L2+ Total Precipitable Water
    standard_name: lwe_thickness_of_atmosphere_mass_content_of_water_vapor
    _Unsigned: true
    valid_range: [ 0 -6]
    scale_factor: 0.00152602
    add_offset: 0.0
    units: mm
    resolution: y: 0.000280 rad x: 0.000280 rad
    coordinates: latitude retrieval_local_zenith_angle quantitative_local_zenith_angle solar_zenith_angle t y x
    grid_mapping: goes_imager_projection
    cell_methods: latitude: point (good quality pixel produced) retrieval_local_zenith_angle: point (good or degraded quality pixel produced) quantitative_local_zenith_angle: point (good quality pixel produced) solar_zenith_angle: point (good quality pixel produced) t: point area: point
    ancillary_variables: DQF_Overall DQF_Retrieval DQF_SkinTemp
unlimited dimensions: 
current shape = (1086, 1086)
filling on

In [18]:
ds.variables['TPW'][:]

masked_array(
  data=[[--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        ...,
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --]],
  mask=[[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],
  fill_value=65535,
  dtype=float32)

También puede ver la descripción de todas las variables ejecutando la tarea **ds.variables.items()**

### Crear proyección

El segundo paso es crear la proyección, es decir, pasar de las coordenadas x e y de GOES a un sistema de longitud-latitud. Para ésto, usaremos la tarea Proj de la librería pyproj.

In [5]:
# Altura de la órbita del satélite
sat_h= ds.variables['goes_imager_projection'].perspective_point_height

# Longitud geográfica de la órbita del satélite
sat_lon = ds.variables['goes_imager_projection'].longitude_of_projection_origin

# Dirección en que se hace el barrido
sat_sweep = ds.variables['goes_imager_projection'].sweep_angle_axis

# Leo las coordenadas en la imagen. 
# Son generadas con una proyección geo-estacionaria

x = ds.variables['x'][:]
y = ds.variables['y'][:]

# map object with pyproj
p = Proj(proj='geos', h=sat_h, lon_0=sat_lon, sweep=sat_sweep)

# Convert map points to latitude and longitude with the magic provided by Pyproj
XX, YY = np.meshgrid(x*sat_h, y*sat_h)
lons, lats = p(XX, YY, inverse=True)

### Definir límites del área a estudiar

Ahora que tenemos latitudes y longitudes, podemos definir más fácilmente los límites del área requerida. En este ejemplo definiremos un área que toma parte de Centro América y el norte de Suramérica, centrada en el límite entre Colombia y Perú. Esto es, entre -90° y -60° de longitud, y entre -15° y 15° de latitud.

**Nota:** En mi experiencia trabajando con estos mapas he notado que los resultados son más "bonitos" si se toma un área cuadrada (en términos de longitud angular).

In [6]:
# Definimos los límites oeste y este en longitud, y sur y norte en latitud, respectivamente
# Cambie estos números si quiere trabajar en un area diferente
bound = {'lon': [-90,-60], 
         'lat': [-15,15]}

xmin,ymin = p(bound['lon'][0],bound['lat'][0])/sat_h
xmax,ymax = p(bound['lon'][1],bound['lat'][1])/sat_h

# Creamos unos arreglos que indican las posiciones en los arreglos de x e y
# que estan dentro del area
sel_x = np.where((x>=xmin) & (x<=xmax))
sel_y = np.where((y>=ymin) & (y<=ymax))

### Seleccionamos los datos dentro del área

In [7]:
x_col = x[sel_x]
y_col = x[sel_y]
TPW = ds.variables['TPW'][sel_y[0].min():sel_y[0].max()+1,sel_x[0].min():sel_x[0].max()+1]

Comparemos el tamaño de las matrices

In [8]:
print('Imagen completa',np.shape(ds.variables['TPW'][:]))
print('Imagen cortada',np.shape(TPW))

Imagen completa (1086, 1086)
Imagen cortada (324, 314)


## 2. Guardar una nueva imagen

Al tomar los datos para una región más pequeña, la cantidad de datos disminuye notablemente, por lo que puede ser factible almacenar una imagen cortada para ahorrar espacio en disco. A continuación haremos la prueba para una imagen 2D (el procedimiento para imágenes 3D es similar, solo hace falta agregar la variable en z, que puede ser presión, por ejemplo).

In [9]:
# Abrimos el archivo a escribir
# El primer argumento debe contener el nombre (path) del archivo
# Lo nombraremos asi TPW_20200512-1700.nc, con el nombre de la variable, la fecha y la hora
new = Dataset('TPW_20200512-1700.nc','w',format='NETCDF4')

# Creamos las dimensiones usando las coordenadas
yy = new.createDimension('y',len(y_col))
xx = new.createDimension('x',len(x_col))

# Agregamos variables usando createVariaible
# Primero va el nombre de la variable, luego el tipo y tercero las dimensiones
# Debe asegurarse de que cada tipo coincida con los mostrados en la imagen original
ys = new.createVariable('y','i2',('y',))
xs = new.createVariable('x','i2',('x',))
var_ = new.createVariable('TPW','i2',('y','x',))
g_ = new.createVariable('goes_imager_projection','i4')
# Note que creamos las variables como enteros para que ocupen menos espacio en disco.
# Para que los valores sean correctos se deben definir los atributos de las variables tal y como en los archivos originales, como se muestra a continuación.


Ahora, debemos definir los atributos de las variables. Como son los mismos de la imagen original, yo los copiaré, aunque, al no guardar todas las variables de la imagen, algunos atributos ya no tendrán sentido (la verdad es que me da pereza escribir a mano los importantes).

Dichos atributos son llamados de forma abreviada *ncattr* y para copiarlos utilizaremos las tareas **setncatts** y **getncattr**

Hagamos esto primero para la variable **goes_imager_projection**.

In [10]:
g_.setncatts({k: ds.variables['goes_imager_projection'].getncattr(k) for k in ds.variables['goes_imager_projection'].ncattrs()})

Ahora para las coordenadas:

In [11]:
ys.setncatts({k: ds.variables['y'].getncattr(k) for k in ds.variables['y'].ncattrs()})
xs.setncatts({k: ds.variables['x'].getncattr(k) for k in ds.variables['x'].ncattrs()})

Y por último para la variable de TPW:

In [12]:
var_.setncatts({k: ds.variables['TPW'].getncattr(k) for k in ds.variables['TPW'].ncattrs()})

Miremos como quedaron

In [13]:
new.variables['goes_imager_projection']

<class 'netCDF4._netCDF4.Variable'>
float32 goes_imager_projection()
    long_name: GOES-R ABI fixed grid projection
    grid_mapping_name: geostationary
    perspective_point_height: 35786023.0
    semi_major_axis: 6378137.0
    semi_minor_axis: 6356752.31414
    inverse_flattening: 298.2572221
    latitude_of_projection_origin: 0.0
    longitude_of_projection_origin: -75.0
    sweep_angle_axis: x
unlimited dimensions: 
current shape = ()
filling on, default _FillValue of 9.969209968386869e+36 used

In [14]:
ys

<class 'netCDF4._netCDF4.Variable'>
float32 y(y)
    scale_factor: -0.00028
    add_offset: 0.1519
    units: rad
    axis: Y
    long_name: GOES Projection y-Coordinate
    standard_name: projection_y_coordinate
unlimited dimensions: 
current shape = (324,)
filling on, default _FillValue of 9.969209968386869e+36 used

In [15]:
var_

<class 'netCDF4._netCDF4.Variable'>
int16 TPW(y, x)
    _FillValue: -1
    long_name: ABI L2+ Total Precipitable Water
    standard_name: lwe_thickness_of_atmosphere_mass_content_of_water_vapor
    _Unsigned: true
    valid_range: [ 0 -6]
    scale_factor: 0.00152602
    add_offset: 0.0
    units: mm
    resolution: y: 0.000280 rad x: 0.000280 rad
    coordinates: latitude retrieval_local_zenith_angle quantitative_local_zenith_angle solar_zenith_angle t y x
    grid_mapping: goes_imager_projection
    cell_methods: latitude: point (good quality pixel produced) retrieval_local_zenith_angle: point (good or degraded quality pixel produced) quantitative_local_zenith_angle: point (good quality pixel produced) solar_zenith_angle: point (good quality pixel produced) t: point area: point
    ancillary_variables: DQF_Overall DQF_Retrieval DQF_SkinTemp
unlimited dimensions: 
current shape = (324, 314)
filling on

### Ahora, asignemos los valores a cada variable

In [16]:
#Asignar valores
ys[:] = y_col
xs[:] = x_col
var_[:] = TPW

### Finalmente, se debe cerrar el archivo

In [17]:
new.close()